## Web Traffic Analytics Lab 

### Retrieve Your Downloaded Chrome Browsing History Data

In [ ]:
# This step assumes you have already downloaded the browsing history via Chrome plug-in

import json
import urlparse
import os

data_path = os.path.expanduser('~') + "/Downloads/chrome_history.json"

with open(data_path, 'rw+') as data_file:
    data = json.load(data_file)

### Check out the Structure of the Data

In [ ]:
#check out the structure of the file
print type(data)
print ' '
print data[0].keys()
print ' '
print data[0]['url']

parsed = urlparse.urlparse(data[0]['url'])
print parsed[1].encode("ascii")

### Create a Web Browsing History Based on the Visited Site's Domain

In [ ]:
n = len(data)
link_list = [urlparse.urlparse(data[k]['url'])[1].encode('ascii') for k in range(n)]

In [ ]:
link_list[:10]

### Create a Visit Frequency Dictionary for Each of Your Domains

In [ ]:
count = dict()
for k in link_list:
    if k:
        count[k] = count.get(k,0) + 1  
        
# sorted(count, key=lambda x: x[1])
# count

In [ ]:
import operator
li_sort = sorted(count.items(), key=operator.itemgetter(1), reverse = True)[:50]
li_sort[:50]

In [ ]:
# from operator import itemgetter, attrgetter, methodcaller

# print sorted(count, key=itemgetter(1)) == sorted(count, key=lambda x: x[1])

In [ ]:
import pandas as pd
%matplotlib inline

urls = pd.DataFrame(li_sort, columns=['url', 'count'])

urls.index = urls['url']

urls.sort('count').plot(kind="barh")

### What Additional Information Can I Obtain from My Browsing History?

In [ ]:
import re
re.sub(r'[^\w]', ' ', urlparse.urlparse(data[560]['url'])[2].encode('ascii'))

In [ ]:
import re
def paras_extract(data):
    w = list()
    for i in range(len(data)): 
        ll = urlparse.urlparse(data[i]['url'])[2].encode('ascii')
        w.append(re.sub(r'[^\w]', ' ', ll))
    return w

In [ ]:
ptext = ''.join(paras_extract(data))
# ptext.split()

# Google Search Terms
# s1 = set(re.findall(r'20\w+', ptext)) 

In [ ]:
ptext[:250]

### Installation of Additional Packages May be Necessary

In [ ]:
# pip install git+git://github.com/amueller/word_cloud.git   
# conda install pil 

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# remove URLs, RTs, and twitter handles
no_urls_no_tags = " ".join([word for word in ptext.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

In [ ]:
wordcloud = WordCloud(
                      stopwords=STOPWORDS,
                      background_color='black',
                      width=2400,
                      height=1800
                     ).generate(no_urls_no_tags)

In [ ]:
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('./my_wordcloud_1.png', dpi=300)
plt.show()

### Preparation for Map Reduce

In [ ]:
#create file compatible with map reduce

with open('web_history.tsv', 'w_') as data_out:
    for d in data: 
        #data_out.write(str(d['url'] + '\t' + str(d['visitCount']) + '\n'))
        data_out.write(str(d['url'] + '\n'))

### Map Reduce Execution

In [ ]:
!cat web_history.tsv | python ./code/mapper.py | sort -k1 | python ./code/reducer.py | sort -nk 2 -r | head -n 15